<a href="https://colab.research.google.com/github/MichelleThomas1200/conv-neural-nets-trial-/blob/main/neural_nets_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hello there

In [1]:
import numpy as np

import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage

import torch
import torchvision

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2


In [2]:
#train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

!pip install kaggle

from google.colab import files 
files.upload()

#Make a directory named kaggle and copy the kaggle.json file there; and change its permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#imagenet_data = torchvision.datasets.ImageNet('https://kaggle.com/saroz014/plant-diseases')
#data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=4, shuffle=True, num_workers=args.nThreads)

#choose location of kaggle.json
#C:\Users\micht\.kaggle

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d saroz014/plant-diseases

100% 6.39G/6.39G [01:09<00:00, 138MB/s]
100% 6.39G/6.39G [01:09<00:00, 99.2MB/s]


In [4]:
from zipfile import ZipFile
file_name = 'plant-diseases.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  #zip.printdir()
  zip.extractall()
  #zip.close()
  print('Done')

Done


In [ ]:
#'dataset_itr2\train\Tomato__Septoria_leaf_spot'
#dataset is in a folder, not a file... so im not sure how to handle this

In [ ]:
# call extract() method to extract any file by specifying its path in the zip file. #e.g.
# zip.extract('python_files/python_wiki.txt')

#If you want to read some specific file, :
# data = zip.read(name_of_file_to_read)

In [ ]:
#to unzip data:
# ! unzip plant-diseases.zip -d plant-diseases

In [1]:
import os
import shutil

!unzip -u "plant-diseases.zip" -d "unzipped-folder"

In [ ]:
# DONT EXECUTE EVER!!  ----- unless u accidentally created 2 copies of folder dataset_itr2
# !rm -rf unzipped-folder/dataset_itr2/dataset_itr2 

In [5]:
!mkdir -p /content/TestFolder
!mkdir -p /content/TrainFolder

In [6]:
import os
from torch import utils
import shutil

dir_path = '/content/dataset_itr2/dataset_itr2/test'
cl = os.listdir(dir_path)
print(cl)
unique_plants = []
for i in cl:
    x = i.split('_')
    if x[0] not in unique_plants:
        unique_plants.append(x[0])
print("Number of Unique Plants: ",len(unique_plants))
print("Unique Plants: ",unique_plants)

#What I wanna do here, is seperate the data based on their unique classes
#Creating Directories in another folder to avoid confusion, so mypath is like the target path:
my_path = '/content/TestFolder'
for u in unique_plants:
    new_dir = my_path + '/' + u
    if os.path.isdir(new_dir) == False:
        os.mkdir(new_dir)
        print(new_dir + ' -> Directory created')

#moving data into them
new_dirs = os.listdir(my_path)
#Note : cl holds the original directories as segregated by disease
for i in cl:
    x = i.split('_')
    for d in new_dirs:
        if x[0] == d:
            #check if folder already exists  
            target = my_path + '/' + d
            if os.path.isdir(target) == True:
                shutil.move(dir_path + '/' + i, target)



dir_path = '/content/dataset_itr2/dataset_itr2/train'
cl = os.listdir(dir_path)
print(cl)
unique_plants = []
for i in cl:
    x = i.split('_')
    if x[0] not in unique_plants:
        unique_plants.append(x[0])
print("Number of Unique Plants: ",len(unique_plants))
print("Unique Plants: ",unique_plants)

#What I wanna do here, is seperate the data based on their unique classes
#Creating Directories in another folder to avoid confusion, so mypath is like the target path:
my_path = '/content/TrainFolder'
for u in unique_plants:
    new_dir = my_path + '/' + u
    if os.path.isdir(new_dir) == False:
        os.mkdir(new_dir)
        print(new_dir + ' -> Directory created')

#moving data into them
new_dirs = os.listdir(my_path)
#Note : cl holds the original directories as segregated by disease
for i in cl:
    x = i.split('_')
    for d in new_dirs:
        if x[0] == d:
            #check if folder already exists  
            target = my_path + '/' + d
            if os.path.isdir(target) == True:
                shutil.move(dir_path + '/' + i, target)

['Tomato___Bacterial_spot', 'Pepper,_bell___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Tomato___Septoria_leaf_spot', 'Blueberry___healthy', 'Squash___Powdery_mildew', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Pepper,_bell___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Peach___Bacterial_spot', 'Tomato___healthy', 'Potato___Late_blight', 'Peach___healthy', 'Tomato___Leaf_Mold', 'Grape___Black_rot', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Cherry_(including_sour)___healthy', 'Apple___Black_rot', 'Apple___healthy', 'Tomato___Target_Spot', 'Soybean___healthy', 'Apple___Apple_scab', 'Raspberry___healthy', 'Tomato___Late_blight', 'Corn_(maize)___Northern_Leaf_Blight', 'Tomato___Early_blight', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Apple___Cedar_apple_rust', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Potato___Early_blight', 'Corn_(mai

In [7]:
from torchvision import transforms
from torchvision import datasets

In [8]:
#import train dataset as tensors
#Sticking to a single species - Cherry
transform =  transforms.Compose([transforms.ToTensor()])

data = datasets.ImageFolder('/content/TrainFolder/Tomato', transform=transform)
#validation and training data can be defined over the 'train' folder itself
split_ratio = 0.3
vSize = int(len(data) * split_ratio)
tSize = len(data) - vSize
train_set, validation_set = utils.data.random_split(data, [tSize, vSize])

#Use Dataloader to load the data into an itterable form
train = utils.data.DataLoader(train_set, batch_size = 64, shuffle = True)
validate = utils.data.DataLoader(validation_set, batch_size = 64, shuffle = True)

In [9]:
# ResNet ?

In [11]:
from fastai.vision import *

In [18]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

import copy
import time

In [30]:
from fastai.callbacks.hooks import num_features_model
#define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH = 'model.pth'

#load data
data_transforms = {

    'TrainFolder/Tomato' : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'TestFolder/Tomato': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),#
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ])
}
data_dir = '/content'

#create custom dataset
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['TrainFolder/Tomato', 'TestFolder/Tomato']}



dataloader = { x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, shuffle = True, num_workers = 4) for x in ['TrainFolder/Tomato', 'TestFolder/Tomato']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['TrainFolder/Tomato', 'TestFolder/Tomato'] }

class_names = image_datasets['TrainFolder/Tomato'].classes
#print(class_names)
#['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


#Training

def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
  since = time.time() #to know time taken to train model
  best_model_wts = copy.deepcopy( model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print("-" * 18)

    #num of epochs for training n validation phase
    for phase in ['TrainFolder/Tomato', 'TestFolder/Tomato']:
      if phase == 'TrainFolder/Tomato':
        model.train() #to train
      else:
        model.eval() #to evaluate model
      
      running_loss = 0.0
      running_corrests = 0

      ##Iterating through data
      for inputs, labels in dataloader[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        #zero-th parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'TrainFolder/Tomato'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          # backward + optimize only if in training phase
          if phase == 'TrainFolder/Tomato':
            loss.backward()
            optimizer.step()

        #generating some stats
        running_loss += loss.item() * inputs.size(0)
        running_corrests += torch.sum(preds == labels.data)

      if phase == 'TrainFolder/Tomato':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrests.double() / dataset_sizes[phase]

      print(' {} Loss: {:.4f} Acc: {:.4f}'.format(phase. epoch_loss, epoch_acc))

      # Deep copy the model
      if phase == 'TestFolder/Tomato' and epoch_acc > best_acc:
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), "./best-model-checkpoint.pt")
        best_acc = epoch_acc

    print()

  time_elapsed = time.time() - since
  print("Training Completed in time {:.0f}m {:.0f}s ".format(time_elapsed//60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  #loading best model weights
  model.load_state_dict(best_model_wts)
  return model


model_ft = models.resnet50(pretrained = True)

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 10) #there r 10 classes in tomato

model_ft = model_ft.to(device)
#Loss function
criterion = nn.CrossEntropyLoss()
#define optimizer
optimizer_ft = optim.Adam(model_ft.parameters, lr = 0.001)

#define scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs = 2)

torch.save(model_ft.state_dict(), './my2epochmodel.pth')

print('\nSaving the model...')
torch.save(model_ft, PATH)

print('Model Saved!')


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored